In [ ]:
train 데이터 전처리-> 분할 -> 모델학습, 평가-> 테스트 데이터 전처리-> 테스트 데이터를 넣고 가격 예측 

In [45]:
import pandas as pd

# 파일을 불러옵니다.
train_data = pd.read_csv('C:/Users/82106/Desktop/제주도 과일 공모전/data/train.csv')


# 전처리 과정
train_data['timestamp'] = pd.to_datetime(train_data['timestamp'])
train_data['year'] = train_data['timestamp'].dt.year
train_data['month'] = train_data['timestamp'].dt.month
train_data['day'] = train_data['timestamp'].dt.day
train_data = pd.get_dummies(train_data, columns=['item', 'corporation', 'location'])
train_data.drop(['ID', 'timestamp'], axis=1, inplace=True)

# 타겟 변수와 특성 변수 분리
X = train_data.drop('price(원/kg)', axis=1)
y = train_data['price(원/kg)']

# XGBoost 회귀 모델 생성 및 훈련
xgb_model = XGBRegressor(objective='reg:squarederror')
xgb_model.fit(X, y)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [49]:
from sklearn.metrics import mean_squared_error
from math import sqrt

# 훈련 데이터와 검증 데이터 분할
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

# 모델을 훈련 세트에 적합시킵니다.
xgb_model.fit(X_train, y_train)

# 검증 세트에 대한 예측을 수행합니다.
valid_predictions = xgb_model.predict(X_valid)

# RMSE 계산
rmse = sqrt(mean_squared_error(y_valid, valid_predictions))
print(f"Validation RMSE: {rmse}")


Validation RMSE: 495.11870773851126


In [53]:
valid_predictions

array([2665.9648   ,   82.56593  ,   -6.957315 , ...,   -4.4644537,
         -9.972278 , 1430.1718   ], dtype=float32)

In [50]:
# test.csv 파일에서 동일한 전처리를 수행
test_data = pd.read_csv('C:/Users/82106/Desktop/제주도 과일 공모전/data/test.csv')
test_data['timestamp'] = pd.to_datetime(test_data['timestamp'])
test_data['year'] = test_data['timestamp'].dt.year
test_data['month'] = test_data['timestamp'].dt.month
test_data['day'] = test_data['timestamp'].dt.day
test_data = pd.get_dummies(test_data, columns=['item', 'corporation', 'location'])
test_data.drop(['ID', 'timestamp'], axis=1, inplace=True)

In [51]:
# test 데이터에 존재하지 않는 컬럼을 train 데이터에서 찾아 0으로 채움
for column in X.columns:
    if column not in test_data.columns:
        test_data[column] = 0

In [52]:
# 예측 수행
test_predictions = xgb_model.predict(test_data[X.columns])

# 예측 결과 확인
print(test_predictions)

[-135.96365   -73.67239   -30.927258 ...  -32.589176  -27.217896
  -25.549486]
